丟一些測試的代碼

In [39]:
import pandas as pd
import numpy as np
import cv2
import os
from PIL import Image
import math
import matplotlib.pyplot as plt

In [ ]:
def rgb2hsv(color):
    color_list=color.tolist()
    b,g,r=color_list[0],color_list[1],color_list[2]
    r, g, b = r/255.0, g/255.0, b/255.0
    mx, mn = max(r, g, b), min(r, g, b)
    m = mx-mn
    if mx == mn:
        h = 0
    elif mx == r:
        if g >= b:
            h = ((g-b)/m)*60
        else:
            h = ((g-b)/m)*60 + 360
    elif mx == g:
        h = ((b-r)/m)*60 + 120
    elif mx == b:
        h = ((r-g)/m)*60 + 240
    if mx == 0:
        s = 0
    else:
        s = m/mx
    v = mx
    return h, s, v


資料庫
===

In [3]:
flower_name_dict={
'tree poppy': '樹罌粟',
'alpine sea holly': '海濱刺芹',
'balloon flower': '桔梗',
'bishop of llandaff': '大麗花',
'black-eyed susan': '黑心金光菊',
'blanket flower': '天人菊',
'californian poppy': '花菱草',
'daisy': '雛菊',
'giant white arum lily': '海芋',
'globe thistle': '藍刺頭',
'iris': '鳶尾花',
'lilac hibiscus' :'朱槿',
'orange dahlia' :'大麗花',
'osteospermum': '藍眼菊',
'pink-yellow dahlia':'',
'poinsettia':'聖誕紅',
'purple coneflower': '紫錐花',
'sunflower': '向日葵',
'thorn apple' :'曼陀羅',
'tree mallow': '花葵'
    
}

In [5]:
import Flower
#Flower.show_match('C:/flowerpic/jpeg-224x224/val/iris/12769.jpeg','C:/flowerpic/jpeg-224x224/val/iris/16301.jpeg')


爬蟲
===

邊緣檢測(?)
---


In [32]:
import cv2
import numpy as np
import imutils
img="flower_pic/tree poppy/12795.jpeg"
image=cv2.imread(img)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#高斯慮波
#v2.GaussianBlur（ SRC，ksize，sigmaX [，DST [，sigmaY [，borderType ] ] ] ）

blurred = cv2.GaussianBlur(gray, (7, 7), 0)

#minVal和maxVal
edged = cv2.Canny(blurred, 50, 200)
cv2.imshow("SIFT", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [33]:
( cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#print("I count {} coins in this image".format(len(cnts)))

coins = image.copy()

cv2.drawContours(coins, cnts, -1, (0, 255, 0), 2)
cv2.imshow("SIFT", coins)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [36]:
for (i, c) in enumerate(cnts):

    (x, y, w, h) = cv2.boundingRect(c)

    print("Part #{}".format(i + 1))

    coin = image[y:y + h, x:x + w]

    cv2.imshow("part", coin)

    mask = np.zeros(image.shape[:2], dtype = "uint8")

    ((centerX, centerY), radius) = cv2.minEnclosingCircle(c)

    cv2.circle(mask, (int(centerX), int(centerY)), int(radius), 255, -1)

    mask = mask[y:y + h, x:x + w]
    cv2.imshow("masked part",cv2.bitwise_and(coin, coin, mask = mask))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
  

Part #1
Part #2
Part #3
Part #4


ANN
===

In [ ]:
import numpy as np
from queue import PriorityQueue

def means(X):
    """
    启发式的选取两个点

    参数
    ----------
    X : 特征矩阵
    
    返回
    ----------
    两个向量点
    """
    iteration_steps = 20
    count = X.shape[0]
    i = np.random.randint(0, count)
    j = np.random.randint(0, count - 1)
    # 保证 i\j 不相同
    j += (j >= i)
    ic = 1
    jc = 1
    p = X[i]
    q = X[j]
    for l in range(iteration_steps):
        k = np.random.randint(0, count)
        di = ic * distance(p, X[k])
        dj = jc * distance(q, X[k])
        if di == dj:
            continue
        if di < dj:
            p = (p * ic + X[k]) / (ic + 1)
            ic = ic + 1
        else:
            q = (q * jc + X[k]) / (jc + 1)
            jc = jc + 1
    return p, q
        
def distance(a, b):
    """
    计算距离

    参数
    ----------
    a : 向量 a

    b : 向量 b
    
    返回
    ----------
    向量 a 与 向量 b 直接的距离
    """
    return np.linalg.norm(a - b)

class annoynode:
    """
    Annoy 树结点
    """
    
    def __init__(self, index, size, w, b, left = None, right = None):
        # 结点包含的样本点下标
        self.index = index
        # 结点及其子结点包含的样本数
        self.size = size
        # 分割超平面的系数
        self.w = w
        # 分割超平面的偏移量
        self.b = b
        # 左子树
        self.left = left
        # 右子树
        self.right = right
    
    def __lt__(self, other):
        # 结点大小比较
        return self.size < other.size

class annoytree:
    """
    Annoy 树算法实现
    
    参数
    ----------
    X : 特征矩阵

    leaf_size : 叶子节点包含的最大特征向量数量，默认为 10
    """
    
    def __init__(self, X, leaf_size = 10):
        def build_node(X_indexes):
            """
            构建结点
            
            参数
            ----------
            X_indexes : 特征矩阵下标
            """
            # 当特征矩阵小于等于指定的叶子结点的大小时，创建叶子结点并返回
            if len(X_indexes) <= leaf_size:
                return annoynode(X_indexes, len(X_indexes), None, None)
            # 当前特征矩阵
            _X = X[X_indexes, :]
            # 启发式的选取两点
            p, q = means(_X)
            # 超平面的系数
            w = p - q
            # 超平面的偏移量
            b = -np.dot((p + q) / 2, w)
            # 构建结点
            node = annoynode(None, len(X_indexes), w, b)
            # 在超平面“左”侧的特征矩阵下标
            left_index = (_X.dot(w) + b) > 0
            if left_index.any():
                # 递归的构建左子树
                node.left = build_node(X_indexes[left_index])
            # 在超平面“右”侧的特征矩阵下标
            right_index = ~left_index
            if right_index.any():
                # 递归的构建右子树
                node.right = build_node(X_indexes[right_index])
            return node
        # 根结点
        self.root = build_node(np.array(range(X.shape[0])))
        
class annoytrees:
    """
    Annoy 算法实现
    
    参数
    ----------
    X : 特征矩阵
    
    n_trees : Annoy 树的数量，默认为 10

    leaf_size : 叶子节点包含的最大特征向量数量，默认为 10
    """
    
    def __init__(self, X, n_trees = 10, leaf_size = 10):
        self._X = X
        self._trees = []
        # 循环的创建 Annoy 树
        for i in range(n_trees):
            self._trees.append(annoytree(X, leaf_size = leaf_size))
            
    def query(self, x, k = 1, search_k = -1):
        """
        查询距离最近 k 个特征向量

        参数
        ----------
        x : 目标向量

        k : 查询邻居数量

        search_k : 最少遍历出的邻居数量，默认为 Annoy 树的数量 * 查询数量
        """
        
        # 创建结点优先级队列
        nodes = PriorityQueue()
        # 先将所有根结点加入到队列中
        for tree in self._trees:
            nodes.put([float("inf"), tree.root])
        if search_k == -1:
            search_k = len(self._trees) * k
        # 待查询的邻居下标数组
        nns = []
        # 循环优先级队列
        while len(nns) < search_k and not nodes.empty():
            # 获取优先级最高的结点
            (dist, node) = nodes.get()
            # 如果是叶子结点，将下标数组加入待查询的邻居中
            if node.left is None and node.right is None:
                nns.extend(node.index)
            else:
                # 计算目标向量到结点超平面的距离
                dist = min(dist, np.abs(x.dot(node.w) + node.b))
                # 将距离做为优先级的结点加入到优先级队列中
                if node.left is not None:
                    nodes.put([dist, node.left])
                if node.right is not None:
                    nodes.put([dist, node.right])
        # 对下标数组进行排序
        nns.sort()
        prev = -1
        # 优先级队列
        nns_distance = PriorityQueue()
        for idx in nns:
            # 过滤重复的特征矩阵下标
            if idx == prev:
                continue
            prev = idx
            # 计算特征向量与目标向量的距离做为优先级
            nns_distance.put([distance(x, self._X[idx]), idx])
        nearests = []
        distances = []
        # 取前 k 个
        for i in range(k):
            if nns_distance.empty():
                break
            (dist, idx) = nns_distance.get()
            nearests.append(idx)
            distances.append(dist)
        return nearests, distances
